# NOSQL - Trabalho prátrico

Adicionar descrição/introdução


## Informação relativa à base de dados

- database:trabalho_pratico
- username: nosql
- password: nosql2024


Identificaram-se 4 coleções: ....

A etapa inicial do trabalho consistiu em estabelecer uma conexão entre o Python e a base de dados Oracle. Isso implicou a especificação do utilizador, palavra-passe, anfitrião, porta e SID do Oracle, previamente definidos. Assim, tornou-se possível executar consultas e recuperar dados da base de dados, possibilitando a visualização de todas as informações lá contidas.

# Librarias e packages necessários

In [17]:
#pip install pymongo
#pip install cx_Oracle
#pip install oracledb
#pip install mysql-connector-python

# Introdução

A base deste trabalho é um conjunto de 17 tabelas que retratam um Hospital. 

O objetivo principal deste trabalho passa por explorar diferentes paradigmas de base de dados, tendo como referências dois tipos de modelos, um modelo relacional e um modelo não relacional. 

Em particular, o modelo não relacional será o mais explorado, sendo que, com auxílio de código (maioritariamente em python) iremos, através de uma estratégia definida e explicada posteriormente, "converter" o modelo relacional já dado em modelos não relacionais explorando as bases de dados MongoDB e Neo4j e seus respetivos paradigmas. 

In [52]:
import oracledb
import getpass
import cx_Oracle

# Pedir senha
#password = getpass.getpass(prompt="Enter Oracle password: ")

# Conectar ao Oracle
#connection = oracledb.connect(user="nosql", password=password, dsn="localhost/xe")
#cursor = connection.cursor()

In [62]:
#MACM1 Rodrigo

connection = oracledb.connect(user="EU", password="12345678", dsn="localhost/ORCLCDB")
cursor = connection.cursor()

 # MONGO

### Pacientes

In [63]:
import pymongo
from pymongo import MongoClient

client = MongoClient("localhost", 27017)
db = client.Hospital

#Patients = db.create_collection("Patients")
#Services = db.create_collection("Services")
#Appointments = db.create_collection("Appointments")
#Staff = db.create_collection("Staff")

collection_P = db.Patients



In [ ]:
#PATIENTS

import mysql.connector

query= "SELECT * FROM PATIENT P, INSURANCE I, MEDICAL_HISTORY MH, EMERGENCY_CONTACT EC WHERE P.POLICY_NUMBER = I.POLICY_NUMBER AND P.IDPATIENT = MH.IDPATIENT AND P.IDPATIENT = EC.IDPATIENT "

cursor.execute(query)
res = cursor.fetchall()
for row in res:
    print(cursor.description)
    document = dict(zip([column[0] for column in res.description], row))
    collection_P.insert_one(document)

cursor.close()
connection.close()
client.close()

In [1]:
#### Parte do Duarte

### Atendimento

In [ ]:
def extract_data_from_oracle():

    # Pedir senha
    password = getpass.getpass(prompt="Enter Oracle password: ")

    # Conectar ao Oracle
    connection = oracledb.connect(user="nosql", password=password, dsn="localhost/xe")
    cursor = connection.cursor()

# Consulta SQL para extrair os dados de HOSPITAL_EPISODE
    sql_episode = """
        SELECT 
            idepisode,
            patient_idpatient
        FROM 
            episode
    """
    cursor.execute(sql_episode)
    episodes = cursor.fetchall()

# Consulta SQL para extrair os dados de HOSPITAL_ROOM
    sql_room = """
        SELECT 
            idroom,
            room_type,
            room_cost
        FROM 
            room
    """
    cursor.execute(sql_room)
    rooms = cursor.fetchall()
 # Consulta SQL para extrair os dados de HOSPITAL_BILL
    sql_bill = """
        SELECT 
            idbill,
            room_cost,
            test_cost,
            other_charges,
            total,
            idepisode,
            registered_at,
            payment_status
        FROM 
            bill
    """
    cursor.execute(sql_bill)
    bills = cursor.fetchall()

    # Consulta SQL para extrair os dados de HOSPITAL_LAB_SCREENING
    sql_lab_screening = """
        SELECT 
            lab_id,
            test_cost,
            test_date,
            idtechnician,
            episode_idepisode
        FROM 
            lab_screening
    """
    cursor.execute(sql_lab_screening)
    lab_screenings = cursor.fetchall()

    # Fechar conexão
    cursor.close()
    connection.close()

    # Inserir os dados na coleção MongoDB
    insert_data_to_mongodb(episodes, rooms, bills, lab_screenings)

In [ ]:
# Função para inserir os dados no MongoDB
def insert_data_to_mongodb(episodes, rooms, bills, lab_screenings):
    
    # Conectar ao MongoDB
    client = MongoClient('mongodb://localhost:27017/')
    db = client['nosql']  # Substitua 'nome_do_banco_de_dados' pelo nome do seu banco de dados no MongoDB
    collection = db['Appointments']

    # Formatar os dados e inserir na coleção MongoDB
    for episode in episodes:
        episode_data = {
            "idepisode": episode[0],
            "patient_idpatient": episode[1],
            # Adicione outros campos conforme necessário
        }
        collection.insert_one(episode_data)

    for room in rooms:
        room_data = {
            "idroom": room[0],
            "room_type": room[1],
            "room_cost": room[2],
            # Adicione outros campos conforme necessário
        }
        collection.insert_one(room_data)

    for bill in bills:
        bill_data = {
            "idbill": bill[0],
            "room_cost": bill[1],
            "test_cost": bill[2],
            "other_charges": bill[3],
            "total": bill[4],
            "idepisode": bill[5],
            "registered_at": bill[6],
            "payment_status": bill[7],
            # Adicione outros campos conforme necessário
        }
        collection.insert_one(bill_data)

    for lab_screening in lab_screenings:
        lab_screening_data = {
            "lab_id": lab_screening[0],
            "test_cost": lab_screening[1],
            "test_date": lab_screening[2],
            "idtechnician": lab_screening[3],
            "episode_idepisode": lab_screening[4],
            # Adicione outros campos conforme necessário
        }
        collection.insert_one(lab_screening_data)

    # Fechar conexão
    client.close()

# Chamar a função principal para extrair dados do Oracle e inseri-los no MongoDB
extract_data_from_oracle()

### 